In [2]:
from bs4 import BeautifulSoup
import requests

import pandas as pd

In [3]:

url = "http://stats.nba.com/game/#!/0041500314/playbyplay/#qtr2"
r  = requests.get(url)

data = r.text

soup = BeautifulSoup(data)

#print(soup.prettify())

# Not workign
table = soup.find('table', attrs={'class': 'no_highlight stats_table"'})

rows = table.findAll('tr')
for tr in rows:
    cols = tr.findAll('td')
    for td in cols:
        text = td.find(text=True) + ';'
        print (text)

ConnectionError: HTTPConnectionPool(host='stats.nba.com', port=80): Max retries exceeded with url: /game/ (Caused by NewConnectionError('<requests.packages.urllib3.connection.HTTPConnection object at 0x10735a978>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known',))

In [4]:


shots_url = 'http://stats.nba.com/stats/playbyplayv2?EndPeriod=10&EndRange=55800&GameID=0041500314&RangeType=2&Season=2015-16&SeasonType=Playoffs&StartPeriod=1&StartRange=0'


# request the URL and parse the JSON
response = requests.get(shots_url)
response.raise_for_status() # raise exception if invalid response
header = response.json()['resultSets'][0]['headers']
shots = response.json()['resultSets'][0]['rowSet']



ConnectionError: HTTPConnectionPool(host='stats.nba.com', port=80): Max retries exceeded with url: /stats/playbyplayv2?EndPeriod=10&EndRange=55800&GameID=0041500314&RangeType=2&Season=2015-16&SeasonType=Playoffs&StartPeriod=1&StartRange=0 (Caused by NewConnectionError('<requests.packages.urllib3.connection.HTTPConnection object at 0x1095305f8>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known',))

In [5]:
# do whatever we want with the shots data
df = pd.DataFrame(shots, columns=header)

pd.set_option('display.max_columns', None)
df.tail(10)

NameError: name 'shots' is not defined

In [146]:
# missing data: propagate score so that every table has the appropriate score

# set first score to 0-0
df.set_value(0,'SCORE',"0 - 0")

# forward-propagate
df.SCORE.fillna(method='ffill', inplace=True)

In [147]:
# timeouts baby

df_home  = df[df.HOMEDESCRIPTION.notnull()]
df_home_timeouts=df_home[df_home.HOMEDESCRIPTION.str.contains("Timeout")]

df_away  = df[df.VISITORDESCRIPTION.notnull()]
df_away_timeouts=df_away[df_away.VISITORDESCRIPTION.str.contains("Timeout")]

In [148]:
total_timeouts = pd.concat([df_home_timeouts, df_away_timeouts])

In [150]:
total_timeouts.sort_values(by="EVENTNUM")[["HOMEDESCRIPTION", "VISITORDESCRIPTION", "SCORE"]]

,HOMEDESCRIPTION,VISITORDESCRIPTION,SCORE
47,None,Warriors Timeout: Regular (Reg.1 Short 0),8 - 15
60,None,Warriors Timeout: Short (Reg.1 Short 1),8 - 22
99,THUNDER Timeout: Regular (Full 1 Short 0),None,19 - 28
142,THUNDER Timeout: Regular (Full 2 Short 0),None,30 - 32
195,None,Warriors Timeout: Regular (Reg.2 Short 1),40 - 47
225,None,Warriors Timeout: Regular (Reg.3 Short 1),43 - 56
274,THUNDER Timeout: Short (Full 2 Short 1),None,50 - 69
307,THUNDER Timeout: Regular (Full 3 Short 1),None,55 - 72
330,THUNDER Timeout: Short (Full 3 Short 2),None,65 - 76
372,None,Warriors Timeout: Regular (Reg.4 Short 1),74 - 84
